<img src = "logo_neuralearn.png" height="100" width= "100" >
 <h1> <center> <font color = "black"> [Deep Learning with TensorFlow 2]   <br><font color="grey"> PARKINSON DISEASE DETECTION </font></center></h1>

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, LayerNormalization, Conv2DTranspose, Dropout, MaxPooling2D, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from IPython.display import Image, display


<H1> DATA PREPARATION </H1>

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, data, batch_size, IN_1, IN_2, shuffle = False):
        
        self.data = data
        self.batch_size = batch_size
        self.data_list = os.listdir(data)
        self.IN_1 = IN_1
        self.IN_2 = IN_2
        
    def __len__(self):
        return int(np.floor(len(self.data_list)/self.batch_size) )

    def __getitem__(self, idx):
        
        X, y = self.__data_generation(idx)
        
        return X,y
    
    def get_spec(self, inp):
    
        zero_padding = (12000 - len(inp))*[0]
        zero_padding = tf.constant(zero_padding, tf.float32)
        inp = tf.cast(inp, tf.float32)
        equal_length = tf.concat([inp, zero_padding], 0)

        spectrogram = tf.signal.stft(
          equal_length, frame_length = 256, frame_step = 127)

        spectrogram = tf.math.log(tf.abs(spectrogram))

        return np.resize(tf.expand_dims(spectrogram, axis = -1), (self.IN_1,self.IN_2,1))


    def __data_generation(self, idx):
        
        X_1, X_2, X_4 = [],[],[]
        y = []
        
        for j in range(idx*self.batch_size, (idx+1)*self.batch_size ):

            data_list = []
            f = open(self.data + self.data_list[idx], "r")
            for x in f:
                data_list.append([int(x) for x in x[:-1].split(';')])
            data_list = np.array(data_list, dtype = float)
            
            data_list[:,0] = (data_list[:,0] - np.mean(data_list[:,0]))/np.std(data_list[:,0])
            data_list[:,1] = (data_list[:,1] - np.mean(data_list[:,1]))/np.std(data_list[:,1])
            data_list[:,3] = (data_list[:,3] - np.mean(data_list[:,3]))/np.std(data_list[:,3])
            
            X_1.append(self.get_spec(tf.constant(data_list)[:,0]))
            X_2.append(self.get_spec(tf.constant(data_list)[:,1]))
            #X_3.append(self.get_spec(tf.constant(data_list)[:,2]))
            X_4.append(self.get_spec(tf.constant(data_list)[:,3]))
            #X_5.append(self.get_spec(tf.constant(data_list)[:,4]))
            #X_6.append(self.get_spec(tf.constant(data_list)[:,5]))
            #X_7.append(self.get_spec(tf.constant(data_list)[:,6]))
            
            if(self.data_list[idx][0] == 'C'):
                y.append(0)
            else:
                y.append(1)
            
            
        return [tf.convert_to_tensor(X_1), tf.convert_to_tensor(X_2),tf.convert_to_tensor(X_4)], tf.convert_to_tensor(y)

In [ ]:
train_images = 'D:/Machine_Learning/datasets/PARKINSON_HW/parkinson/OTHER/'
IN_1, IN_2 = 64, 64
BATCH_SIZE = 2
EPOCH = 1000

In [ ]:
train_gen = DataGenerator(train_images, BATCH_SIZE, IN_1, IN_2)

<h1>MODELING</h1>

In [ ]:
class ConvLayers(tf.keras.Model):

    def __init__(self, NUM_FILTERS):
        super(ConvLayers, self).__init__()
        self.conv = Conv2D(NUM_FILTERS, 5, padding = 'same', activation ='relu',)
        self.pool = MaxPooling2D(strides = 2)
        
    def call(self, x, ):
        
        x = self.pool(self.conv(x))
        
        return x

In [ ]:
inputs = [tf.keras.Input(shape=(IN_1, IN_2, 1)) for _ in range(3)]

x_out = []

for i in range(3):

    
    #x = tf.keras.layers.experimental.preprocessing.Normalization()(inputs[i])
    
    x = ConvLayers(32)(inputs[i])
    x = ConvLayers(64)(x)
    x = Flatten()(x)
    
    x_out.append(x)
    

x_out = tf.concat(x_out, axis = -1)

x_out = Dense(16, activation = 'relu')(x_out)
x_out = Dense(1, activation = 'sigmoid')(x_out)

model = tf.keras.Model(inputs = inputs, outputs = x_out)
model.summary()

<H1> TRAINING AND OPTIMIZATION </H1>

In [ ]:
LR = 1e-3
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = Adam(lr = LR,),
    metrics = 'accuracy',
    #run_eagerly = True,
)

In [ ]:
checkpoint_filepath = 'parkinson/cnn_dense.hdf5'

callback = tf.keras.callbacks.ModelCheckpoint(
    
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

# Train model on dataset
history = model.fit(train_gen, shuffle = True, epochs = EPOCH, callbacks = [callback])

<H1> TESTING </H1>

In [ ]:
test_image = 'D:/Machine_Learning/datasets/PARKINSON_HW/parkinson/OTHER/C_0012.txt'#C_0013

X_1, X_2, X_4 = [], [], []

data_list = []

def get_spec(inp):

    zero_padding = (12000 - len(inp))*[0]
    zero_padding = tf.constant(zero_padding, tf.float32)
    inp = tf.cast(inp, tf.float32)
    equal_length = tf.concat([inp, zero_padding], 0)

    spectrogram = tf.signal.stft(
      equal_length, frame_length = 256, frame_step = 127)

    spectrogram = tf.math.log(tf.abs(spectrogram))

    return np.resize(tf.expand_dims(spectrogram, axis = -1), (IN_1,IN_2,1))

    
f = open(test_image, "r")
for x in f:
    data_list.append([int(x) for x in x[:-1].split(';')])
data_list = np.array(data_list, dtype = float)
            
data_list[:,0] = (data_list[:,0] - np.mean(data_list[:,0]))/np.std(data_list[:,0])
data_list[:,1] = (data_list[:,1] - np.mean(data_list[:,1]))/np.std(data_list[:,1])
data_list[:,3] = (data_list[:,3] - np.mean(data_list[:,3]))/np.std(data_list[:,3])
            
X_1.append(get_spec(tf.constant(data_list)[:,0]))
X_2.append(get_spec(tf.constant(data_list)[:,1]))
X_4.append(get_spec(tf.constant(data_list)[:,3]))

X = [tf.constant(X_1), tf.constant(X_2), tf.constant(X_4)]
model.predict(X)